In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sqlalchemy
# Connect to database
PWD='root'
USR='root'
SQLALCHEMY_DATABASE_URI = 'mysql+pymysql://{}:{}@localhost:3306/gdb0041'.format(USR, PWD)
engine = sqlalchemy.create_engine(SQLALCHEMY_DATABASE_URI)

**P&L Statement**<br />
Gross = 50<br />
pre-Invoice deduction = 5(for regular customer)<br />
Net-Invoice-sales = 45<br />
post-Invoice deduction = 5(display priority fee)<br />
Net-Sales = 40<br />
COGS = 20 (COST OF GOODS SOLD)<br />
Margin = 20<br />
margin % = 50 (M/NS)<br />

Find the croma customer aggregated by product in monthly basis

In [4]:
query = """
select * from dim_customer 
"""
df_dim_customer = pd.read_sql(query,engine)
#df_dim_customer['customer'].unique()
df_dim_customer[df_dim_customer['customer'] == 'Croma']

,customer_code,customer,platform,channel,market,sub_zone,region
48,90002002,Croma,Brick & Mortar,Retailer,India,India,APAC


To create a fiscal year calendar, add 4 months to the calendar date in the database where September marks the fiscal year's start

In [5]:
query = """
select *,
(CASE when Month(date) in (9,10,11) Then 'Q1'
     when Month(date) in (12,1,2) Then 'Q2'
     when Month(date) in (3,4,5) Then 'Q3' else 'Q4' END ) AS Quater
 from fact_sales_monthly
where customer_code = 90002002 and
year(date_add(date,interval 4 month)) = 2021 and
(CASE when Month(date) in (9,10,11) Then 'Q1'
     when Month(date) in (12,1,2) Then 'Q2'
     when Month(date) in (3,4,5) Then 'Q3' else 'Q4' END )='Q1'
"""
df_fact_sales_monthly = pd.read_sql(query,engine)
print(df_fact_sales_monthly.tail())

           date product_code  customer_code  sold_quantity Quater
663  2020-10-01  A7321160301       90002002            479     Q1
664  2020-09-01  A7321160302       90002002            507     Q1
665  2020-10-01  A7321160302       90002002            243     Q1
666  2020-09-01  A7321160303       90002002            190     Q1
667  2020-10-01  A7321160303       90002002            586     Q1


In [6]:
query = """
select m.date,m.product_code,
       p.product,p.variant,m.sold_quantity,
       gp.gross_price,round(m.sold_quantity*gp.gross_price,2) as Total_gp
 from fact_sales_monthly m
 join dim_product p on p.product_code = m.product_code
 join fact_gross_price gp on gp.product_code = m.product_code and gp.fiscal_year = get_fiscal_year(m.date)
where customer_code = 90002002 and
get_fiscal_year(date) = 2021;
"""
fact_sales_monthly = pd.read_sql(query,engine)
print(fact_sales_monthly.tail())

            date product_code          product  variant  sold_quantity  \
3001  2021-02-01  A7321160303  AQ Wi Power Dx3  Premium            186   
3002  2021-04-01  A7321160303  AQ Wi Power Dx3  Premium            148   
3003  2021-05-01  A7321160303  AQ Wi Power Dx3  Premium            487   
3004  2021-06-01  A7321160303  AQ Wi Power Dx3  Premium            289   
3005  2021-08-01  A7321160303  AQ Wi Power Dx3  Premium            158   

      gross_price  Total_gp  
3001      42.8483   7969.78  
3002      42.8483   6341.55  
3003      42.8483  20867.12  
3004      42.8483  12383.16  
3005      42.8483   6770.03  


Croma month wise Total Sales Amount

In [7]:
query = """
select m.date,
       sum(round(m.sold_quantity*gp.gross_price,2)) as Total_gp
 from fact_sales_monthly m
 join fact_gross_price gp on gp.product_code = m.product_code and gp.fiscal_year = get_fiscal_year(m.date)
where customer_code = 90002002 
group by m.date
order by m.date desc;
"""
fact_sales_monthly = pd.read_sql(query,engine)
print(fact_sales_monthly.tail())

          date   Total_gp
34  2018-02-01  144799.54
35  2018-01-01  127574.73
36  2017-12-01  245673.84
37  2017-10-01  162687.56
38  2017-09-01  122407.57


In [11]:
query = """
with cte as(
select m.date,Year(adddate(m.date,interval 4 month)) as fiscal_year,m.product_code,
       p.product,c.market,p.variant,m.sold_quantity,
       gp.gross_price,round(m.sold_quantity*gp.gross_price,2) as Total_gp,
       pd.pre_invoice_discount_pct,pod.discounts_pct+pod.other_deductions_pct as po_discount
 from fact_sales_monthly m
 join dim_product p on p.product_code = m.product_code
 join dim_customer c on c.customer_code = m.customer_code
 join fact_gross_price gp on gp.product_code = m.product_code and gp.fiscal_year = get_fiscal_year(m.date)
 join fact_pre_invoice_deductions pd on pd.customer_code = m.customer_code and pd.fiscal_year = get_fiscal_year(m.date)
 join fact_post_invoice_deductions pod on pod.customer_code = m.customer_code and get_fiscal_year(pod.date) =get_fiscal_year(m.date)
where 
get_fiscal_year(m.date) = 2021)
select *,
round((1-pre_invoice_discount_pct)*Total_gp,2) as Net_Invoice_Sales
from cte 
order by 1 desc
limit 10000 ;

"""
top_customer = pd.read_sql(query,engine)
print(top_customer.tail())

            date  fiscal_year product_code          product    market  \
9995  2021-08-01         2021  A7321160301  AQ Wi Power Dx3  Pakistan   
9996  2021-08-01         2021  A7321160301  AQ Wi Power Dx3  Pakistan   
9997  2021-08-01         2021  A7321160301  AQ Wi Power Dx3  Pakistan   
9998  2021-08-01         2021  A7321160301  AQ Wi Power Dx3  Pakistan   
9999  2021-08-01         2021  A7321160301  AQ Wi Power Dx3  Pakistan   

       variant  sold_quantity  gross_price  Total_gp  \
9995  Standard             23      40.7954    938.29   
9996  Standard             23      40.7954    938.29   
9997  Standard             23      40.7954    938.29   
9998  Standard             23      40.7954    938.29   
9999  Standard             23      40.7954    938.29   

      pre_invoice_discount_pct  po_discount  Net_Invoice_Sales  
9995                    0.2471       0.3037             706.44  
9996                    0.2471       0.3938             706.44  
9997                    0.247

In [12]:
top_customer['Net_Sales'] =  round((1-top_customer['po_discount'])*top_customer['Net_Invoice_Sales'],2)
top_customer[top_customer['fiscal_year'] == 2021]
top_customer.head()

,date,fiscal_year,product_code,product,market,variant,sold_quantity,gross_price,Total_gp,pre_invoice_discount_pct,po_discount,Net_Invoice_Sales,Net_Sales
0,2021-08-01,2021,A7321160301,AQ Wi Power Dx3,India,Standard,681,40.7954,27781.67,0.2061,0.4142,22055.87,12920.33
1,2021-08-01,2021,A7321160301,AQ Wi Power Dx3,India,Standard,681,40.7954,27781.67,0.2061,0.4233,22055.87,12719.62
2,2021-08-01,2021,A7321160301,AQ Wi Power Dx3,India,Standard,681,40.7954,27781.67,0.2061,0.3813,22055.87,13645.97
3,2021-08-01,2021,A7321160301,AQ Wi Power Dx3,India,Standard,681,40.7954,27781.67,0.2061,0.3879,22055.87,13500.40
4,2021-08-01,2021,A7321160301,AQ Wi Power Dx3,India,Standard,681,40.7954,27781.67,0.2061,0.4370,22055.87,12417.45


In [13]:
g = top_customer.groupby('market')['Net_Sales'].sum()
g

market
India        40511375.29
Indonesia    11787858.08
Japan         2772671.23
Pakistan       446715.67
Name: Net_Sales, dtype: float64

In [14]:
g = top_customer.groupby('variant')['Net_Sales'].sum()
g

variant
Standard    55518620.27
Name: Net_Sales, dtype: float64